In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from keras import layers
from stacked_mnist import StackedMNISTData, DataMode

In [3]:
input_img = keras.Input(shape=(28, 28, 1))

# Encoder
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(input_img)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(2, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(2, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((1, 1), padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
encoded = layers.Dense(4, activation="relu")(x)

# Decoder
decoder_input = layers.Dense(128, activation="relu")(encoded)
x = layers.Dense(7*7*2, activation="relu")(decoder_input)
x = layers.Reshape((7, 7, 2))(x)
x = layers.Conv2DTranspose(4, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(8, (3, 3), strides=2, activation="relu", padding="same")(x)
decoded = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.save_weights('initial_model.h5')
autoencoder.summary()

2022-02-24 14:42:59.148816: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 2)         290       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 2)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 2)           38    

In [4]:
generator = StackedMNISTData(mode=DataMode.MONO_BINARY_COMPLETE, default_batch_size=2048)

x_train, y_train = generator.get_full_data_set(training=True)
x_test, y_test = generator.get_full_data_set(training=False)
# "Translate": Only look at "red" channel; only use the last digit. Use one-hot for labels during training
x_train = x_train[:, :, :, [0]]
x_test = x_test[:, :, :, [0]]

In [5]:
autoencoder.fit(x_train, x_train,
                epochs=30,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/30
469/469 [==============================] - 35s 73ms/step - loss: 0.3379 - val_loss: 0.2573
Epoch 2/30
413/469 [=========================>....] - ETA: 3s - loss: 0.2536

KeyboardInterrupt: 